## Подключение библиотек ##

In [ ]:
import time
import numpy as np
import torch
import warnings
import pandas as pd
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from scipy.special import gamma,erfc
from scipy.integrate import quad
from sklearn.metrics import mean_absolute_error
from pyfod.fod import caputo
from functools import partial
import matplotlib.pyplot as plt
%matplotlib inline

## Настройка СUDA ##  

In [ ]:
if torch.cuda.is_available(): 
    dev = "cuda:0" 
else: 
    dev = "cpu" 
device = torch.device(dev) 
device

## Функция Миттаг-Леффлера ##

In [ ]:
def MLF(z, alpha, N=10000):
    # tic = time.perf_counter()

    # Преобразование типов данных
    if not isinstance(z, np.ndarray):
        if isinstance(z, torch.Tensor):
            z = z.numpy()
        else:
            z = np.array(z)
    z = np.atleast_1d(z)
    
    # Случаи для различных параметров alpha
    if alpha == 0:
        return 1/(1 - z)
    elif alpha == 1:
        return np.exp(z)
    elif alpha == 1/2:
        return np.exp(z ** 2) * erfc(-z)
    elif alpha > 1 or all(z > 0):
        k = np.arange(N)
        return np.polynomial.polynomial.polyval(z, 1 / gamma(alpha * k + 1))
        # j = np.arange(N).reshape(-1, 1)
        # E = (np.power(z, j)) / gamma(alpha * j + 1)
        # return torch.tensor(np.sum(E, axis=0))

    def _MLF(z, alpha):
        if z < 0:
            def f(x): return ((np.exp(-x * np.power(-z, 1 / alpha)) * np.power(x, alpha - 1) * np.sin(np.pi * alpha))
                    / (np.power(x, 2 * alpha) + 2 * np.power(x, alpha) * np.cos(np.pi * alpha) + 1))
            return 1 / np.pi * quad(f, 0, np.inf)[0]
        elif z == 0:
            return 1
        else:
            return MLF(z, alpha)

    # toc = time.perf_counter()
    # print(f"Time = {toc-tic}")
    return np.vectorize(_MLF)(z, alpha)


res = MLF(1, 1)
print(f"MLF = {res}, type={type(res)}")


In [ ]:
%%script echo skipping
x = np.linspace(-10, 1, 100)
plt.grid()
for i in range(1, 11):
    plt.plot(x, MLF(x, i/10), label="alpha = "+str(i/10))
plt.legend()
plt.ylim(-2, 5)
plt.xlim(x[0], x[-1])

In [ ]:
%%script echo skipping
x = np.linspace(-10, 1, 100)
plt.grid()
plt.plot(x, MLF(x, 0.1), label="alpha = 0.1")
plt.legend()
plt.ylim(-2, 5)
plt.xlim(x[0], x[-1])

## Параметры задач ##

In [ ]:
lambda_lin = 0.5
y0_lin = 1
m_lin = 1

lambda_nonlin = 0.5
y0_nonlin = 0
m_nonlin = 0.5


#### Точные решения ####

In [ ]:
def solution_lin(x, alpha):
    solution = y0_lin * ml(lambda_lin * np.power(x, alpha), alpha)
    return solution

def solution_nonlin(x, alpha):
    gm = alpha / (1 - m_nonlin)
    tmp = gamma(gm + 1) / (lambda_nonlin * gamma(gm - alpha + 1))
    B = np.power(tmp, 1 / (m_nonlin - 1))
    solution = B * np.power(x, gm)
    return solution

In [ ]:
Equations = {"Linear" : (y0_lin, lambda_lin, m_lin, solution_lin), "Nonlinear" : (y0_nonlin, lambda_nonlin, m_nonlin, solution_nonlin)}
Equations


In [ ]:
# x = torch.linspace(0, 1, 1000)
# plt.grid()
# plt.plot(x, Equations['Linear'][3](x, 0.1))
# plt.plot(x, Equations['Nonlinear'][3](x, 0.1))
# plt.legend(Equations.keys())

## Модель нейронной сети ##

In [ ]:
class Net(nn.Module):
    def __init__(self, num_hidden, size_hidden, activation=nn.Tanh()):
        super(Net, self).__init__()
        self.activation_function = activation

        self.layer_input = nn.Linear(1, size_hidden).double()
        self.hidden_layers = nn.ModuleList(
            [nn.Linear(size_hidden, size_hidden).double() for _ in range(num_hidden - 1)])
        self.layer_output = nn.Linear(size_hidden, 1).double()

    def forward(self, x):
        x = x.type(torch.double)
        x = self.layer_input(x)
        x = self.activation_function(x)
        for layer in self.hidden_layers:
            x = layer(x)
            x = self.activation_function(x)
        output = self.layer_output(x)
        return output


### Инициализация весов ###

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)

### Текущий результат работы сети ###

In [ ]:
def net_fn(nn, x):
    return nn(x)

### Вычисление производной ###

In [ ]:
def df(nn, alpha, x):
    def f(x):
        x = torch.from_numpy(x)
        return net_fn(nn, x)
    result = torch.zeros(x.shape)
    for k in range(len(x)):
        fd = caputo(f=f, alpha=alpha, lower=0, upper=x[k], quadrature='rs', n=100)
        result[k] = fd['fd']
    return result

### Функция ошибки ###

In [ ]:
def loss_function(nn, alpha, x=None, verbose=False):
    lin = problem
    mse_loss = torch.nn.MSELoss()

    # Начальное условие
    initial_condition = Equations[lin][0]
    initial_loss = net_fn(nn, x[0]) - initial_condition
    initial_loss = mse_loss(initial_loss, torch.zeros_like(initial_loss))
    
    # Внутренняя ошибка
    lmbd = Equations[lin][1]
    m = Equations[lin][2]
    
    interior_loss = df(nn, alpha, x[1:]) - lmbd * torch.pow(torch.abs(net_fn(nn, x[1:])), m)  
    interior_loss = mse_loss(interior_loss, torch.zeros_like(interior_loss))

    # print(f"Initial loss: {initial_loss}")
    # print(f"Interior loss: {interior_loss}")
    
    loss = initial_loss + interior_loss
    return loss


### Цикл обучения ###

In [ ]:
def training_loop(nn, alpha, loss_fn, optimizer, scheduler=None, n_epochs=1000):
    train_loss = torch.zeros(n_epochs)
    
    def closure():
        if torch.is_grad_enabled():
            optimizer.zero_grad()
        loss = loss_fn(nn, alpha)
        if loss.requires_grad:
            loss.backward()
        return loss
    
    for epoch in range(1, n_epochs + 1):
                       
        if isinstance(optimizer, torch.optim.LBFGS):
            optimizer.step(closure)
            loss = closure()
        else:
            loss = loss_fn(nn, alpha)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()   
            
        if scheduler is not None:
            scheduler.step(loss)
            
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, learning rate = {optimizer.param_groups[-1]['lr']}, loss={loss},\n")
                 
    return nn, train_loss      


## Обучение сети ##

### Гиперпараметры сети ###

In [ ]:
problem = 'Nonlinear'
num_hidden = 1
size_hidden = 60
activation_function = nn.Tanh()

NN = Net(num_hidden, size_hidden, activation_function)
NN.apply(init_weights)

learning_rate_Adam = 0.01
learning_rate_LBFGS = 0.01
epochs = 1000
alpha = 0.1

optimizer_adam = torch.optim.AdamW(NN.parameters(), lr=learning_rate_Adam, weight_decay=0.00001)
scheduler_adam = ReduceLROnPlateau(optimizer_adam, 'min', factor=0.1, patience=1000, min_lr=1e-6)

optimizer_lbfgs = torch.optim.LBFGS(NN.parameters(), lr=learning_rate_LBFGS)


### Тренировка на заданном интервале ###

In [ ]:
x = torch.linspace(0, 1, 10).reshape(-1, 1)
loss_fn = partial(loss_function, x=x, verbose=True)

PINN_Adam, train_loss_Adam = training_loop(
    nn=NN,
    alpha=alpha,
    loss_fn=loss_fn,
    optimizer=optimizer_adam,
    scheduler=scheduler_adam,
    n_epochs=epochs
)


In [ ]:
# PINN_LBFGS, train_loss_LBFGS = training_loop(
#     nn=PINN_Adam,
#     alpha=alpha,
#     loss_fn=loss_fn,
#     optimizer=optimizer_lbfgs,
#     n_epochs=1000
# )

In [ ]:
PINN = PINN_Adam
x_eval = torch.linspace(0, 1, 1000).reshape(-1, 1)
nn_approx = PINN(x_eval).detach().numpy()
sol = Equations[problem][3](x_eval, alpha)

### Оценка качества модели ###

In [ ]:
def error(solution, approximation):
    if isinstance(solution, torch.Tensor):
        solution = solution.detach().numpy()
    if np.any(np.isnan(approximation)):
        return np.nan, np.nan, np.nan
    MAE = mean_absolute_error(solution[1:], approximation[1:])
    MRE = np.mean(np.fabs(approximation[1:] - solution[1:]) / np.fabs(solution[1:])) * 100
    InitLoss = float(np.fabs(solution[0] - approximation[0]))
    # print(f"Mean absolute error (MAE) = {MAE:.5}\nMean relative error (MRE) = {MRE:.5}%\nLoss on initial condition = {InitLoss:.5}")
    return MAE, MRE, InitLoss

# MAE, MRE, InitLoss = error(sol, nn_approx)


### Отрисовка графиков ###

In [ ]:
def plot_graph(x, solution, approximation, name=None):
    plt.figure(figsize=(6, 6))
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xlim(-0.1, 1.1)
    
    plt.plot(x, approximation, '-.', label='PINN')
    plt.plot(x, solution, label='Solution')
    plt.grid()
    plt.legend()

    MAE, MRE, InitLoss = error(solution, approximation)
    plt.title(f"Mean absolute error (MAE) = {MAE:.5}\nLoss on initial condition = {InitLoss:.5}")
    # plt.text(0.4, solution[0], f"Mean absolute error (MAE) = {MAE:.5}\nLoss on initial condition = {InitLoss:.5}",
    #          bbox={"fill": False})

    if name is not None:
        plt.savefig(f"{name}.png")
    
    # plt.cla()
    return MAE, MRE, InitLoss
    
# plot_graph(x_eval, sol, nn_approx, train_loss_Adam)

## Тесты ##

In [ ]:
%%script echo skipping


problem = 'Nonlinear'
N = 1
num_hidden = 1
size_hidden = 80

activation_function = nn.Tanh()
learning_rate_Adam = 0.01
epochs = 40000

alphas = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
x_eval = torch.linspace(0, 1, 1000).reshape(-1, 1)

In [ ]:
%%script echo skipping

columns = {"Hidden layers": [], "Neurons in HL": [], "Activation function": [],
                        "Learning rate": [], "N_x": [], "Epochs": [], "Alpha": [], "MAE": [],
                        "InitLoss": [], "Time" : []}
Test_data = pd.DataFrame(columns)

In [ ]:
%%script echo skipping

warnings.simplefilter(action='ignore', category=FutureWarning)
Test = 1
x = torch.linspace(0, 1, 40).reshape(-1, 1)
loss_fn = partial(loss_function, x=x, verbose=True)
for alpha in alphas:
    MAE = np.zeros(N)
    InitLoss = np.zeros(N)
    Time = np.zeros(N)
    for k in range(N):
        
        NN = Net(num_hidden, size_hidden, activation_function)
        NN.apply(init_weights)
        
        optimizer_adam = torch.optim.AdamW(NN.parameters(), lr=learning_rate_Adam, weight_decay=0.00001) 
        scheduler = ReduceLROnPlateau(optimizer_adam, 'min', factor=0.1, patience=4000, min_lr=1e-6)
        
        tic = time.perf_counter()
        PINN_Adam, train_loss_Adam = training_loop(
            nn=NN,
            alpha=alpha,
            loss_fn=loss_fn,
            optimizer=optimizer_adam,
            scheduler=scheduler,
            n_epochs=epochs
        )
        toc = time.perf_counter()
        Time[k] = toc - tic
        
        PINN = PINN_Adam
        nn_approx = PINN(x_eval).detach().numpy()
        sol = Equations[problem][3](x_eval, alpha)
        MAE[k], MRE, InitLoss[k] = plot_graph(x_eval, sol, nn_approx, f"{alpha}_{k}")
        torch.save(PINN_Adam, f"{problem}_alpha_{alpha}_{k}.pth")
        
    tmp = {"Hidden layers": num_hidden, "Neurons in HL": size_hidden, "Activation function": "Tanh",
                        "Learning rate": learning_rate_Adam, "N_x": 40, "Epochs": epochs, "Alpha": alpha, "MAE": MAE.mean(),
                        "InitLoss": InitLoss.mean(), "Time" : Time.mean()}
    Test_data = pd.concat([Test_data, pd.DataFrame.from_records([tmp])], ignore_index=True)
    Test_data.to_excel(f"Alphas_{problem}_{N}_realizations.xlsx")
    print(f"Test {Test} completed.\n")
    Test += 1